In [3]:
# Calculation of Exomoon Radio Flux  
# Authors: Apurva Oza, Kaustubh Hakim

import numpy as np
from astropy import units as u
u.imperial.enable()
from astropy.constants import mu0, N_A, R_jup, GM_jup
# amu = 1 / N_A * u.g / u.mol  # 1 / Avogadro's no.
import matplotlib.pyplot as plt

In [4]:
# Radio Flux, equations 4 and 8, Noyola et al. 2014 ApJ 791
def radio_flux (d = 19.3, nu_c = 140, B_p = 50, a_s = 6, rho_s = 1.96e-14, v0 = 57e5, r_s = 1):

    d      = d * u.pc.cgs             # d: distance to the object (parsec)
    nu_c   = nu_c * u.MHz.cgs         # nu_c: cyclotron frequency (MHz)
    B_p    = B_p * u.Gauss            # B_p: planetary magnetic field (Gauss)
    a_s    = a_s * u.Unit(1)          # a_s: ratio of semimajor axis of satellite to planet radius
    rho_s  = rho_s * u.g * (u.cm)**-3 # rho_s: satellite plasma density (g/cm3), Io = 7e-20 g/cm3 
    v0     = v0 * u.cm / u.s          # v0: plasma speed relative to satellite
    beta_s = 0.01                     # beta_s: efficiency coefficient
    omega  = 0.2                      # omega: soild angle through which source power is emitted
    r_s    = r_s * u.Unit(1)          # r_s: ratio of radius of satellite to radius of Io
    
    r_io   = 1821e5 * u.cm            # r_io: radius of Io (cm)
    del_nu = nu_c / 2                 # del_nu: signal bandwidth (MHz)
    B_s    = B_p / a_s**3             # B_s: planetary magnetic field at the location of satellite (Gauss)
    
    rho_c  = (1/mu0 * (B_s / v0)**2).decompose().cgs   # rho_c: critical density
    fraction = np.sqrt(rho_s / (rho_s + rho_c ))
    
    # P_s: Satellite radio power or maximum radio emission intensity 
    P_s  = (np.pi * beta_s * (r_s*r_io)**2 * B_s**2 * v0 / mu0 * fraction).decompose().cgs
    
    # S_nu: Radio flux at the location of observer
    S_nu = (P_s / (omega * del_nu * d**2)).to('mJy').round(4)
    
    return S_nu

In [5]:
# Plasma Density, Oza et al. 2019 ApJ 885
def plasma_density (Mdot = 10**8.4, t = 1140, r_p = 1.13, m_p = 1.13, a_s = 6, v_ej = 2e5):
    
    Mdot   = Mdot * u.kg.cgs / u.s             # Mdot: mass loss rate (kg/s)
    t      = t * u.s                           # t: ionization timescale (s)
    r_p    = r_p * u.Unit(1)                   # r_p: ratio of planet radius to jupiter radius 
    m_p    = m_p * u.Unit(1)                   # m_p: ratio of planet mass to jupiter mass 
    a_s    = a_s * u.Unit(1)                   # a_s: ratio of semimajor axis of satellite to planet radius
    v_ej   = v_ej * u.cm / u.s                 # v_ej: satellite outgassing velocity  (cm/s)
    
    d_orb  = (a_s * r_p * R_jup).cgs           # d_orb: satellite orbital distance (cm)
    v_orb  = np.sqrt(GM_jup * m_p / d_orb).cgs # v_orb: satellite orbital velocity (cm/s)
    
    # Vtorus: volume of torus
    Vtorus = 2.0 * np.pi**2 * d_orb**3 * (v_ej/v_orb)**2
    
    # rho_s: plasma density
    rho_s  = (Mdot * t / Vtorus).cgs
    
    return rho_s

In [61]:
# WASP 49 b (exoplanet archive - Stassun et al. 2017)

# Fixed parameters for this object. Do not modify
p_WASP49  = 2.782   # orbital period (days)
d_WASP49  = 194.55  # distance to the object (parsec) 
m_WASP49  = 0.399   # ratio of planet mass to jupiter mass 
r_WASP49  = 1.198   # ratio of planet radius to jupiter radius

# Following parameters based on GMRT frequencies
B_WASP49  = 150/2.8 # planetary magnetic field (Gauss)
nu_WASP49 = 150     # cyclotron frequency

# Following parameters can be tweaked
rs_WASP49 = 1       # ratio of satellite radius to Io radius (Earth radius = 3.5 * Io radius)
a_WASP49  = 1.127 # 1.74 # 1.127    # ratio of semimajor axis of satellite to planet radius

# Plasma parameters
t_WASP49  = 241.36     # ionization timescale (s)
Md_WASP49 = 10**5 * 1000 # mass loss rate (kg/s)

# Calculation of plasma density
rho_WASP49 = plasma_density(Mdot=Md_WASP49, t = t_WASP49, a_s = a_WASP49, r_p = r_WASP49, m_p = m_WASP49).value

# Calculation of radio flux
S_WASP49  = radio_flux(d = d_WASP49, nu_c = nu_WASP49, B_p = B_WASP49, r_s = rs_WASP49, a_s = a_WASP49, \
                         rho_s = rho_WASP49)

print('WASP 49 b:  S =', S_WASP49, 'at rho_s =', rho_WASP49, ' g/cm3, B_p =', \
      B_WASP49, 'G and f =', nu_WASP49, 'MHz')

WASP 49 b:  S = 1.0804 mJy at rho_s = 1.780025312263324e-16  g/cm3, B_p = 217.85714285714286 G and f = 610 MHz


In [69]:
# HD 189733 b (exoplanet archive - Addison et al. 2019)

# Fixed parameters for this object. Do not modify
p_HD189  = 2.219   # orbital period (days)
d_HD189  = 19.76   # distance to the object (parsec) 
m_HD189  = 1.166   # ratio of planet mass to jupiter mass 
r_HD189  = 1.119   # ratio of planet radius to jupiter radius

# Following parameters based on GMRT frequencies
B_HD189  = 150/2.8 # planetary magnetic field (Gauss): ~ 20-70 G (Cauley et al., Yadav et al.)
nu_HD189 = 150     # cyclotron frequency

# Following parameters can be tweaked
rs_HD189 = 1       # ratio of satellite radius to Io radius (Earth radius = 3.5 * Io radius) # changed in Mdot
a_HD189  = 1.6    # 2.1 # 1.6     # ratio of semimajor axis of satellite to planet radius

# Plasma parameters
t_HD189  = 1011    # ionization timescale (s)
Md_HD189 = 10**5.4 * 1000 # mass loss rate (kg/s)

# Calculation of plasma density
rho_HD189 = plasma_density(Mdot = Md_HD189, t = t_HD189, a_s = a_HD189, r_p = r_HD189, m_p = m_HD189).value

# Calculation of radio flux using calculated and fixed plasma density
S_HD189  = radio_flux(d = d_HD189, nu_c = nu_HD189, B_p = B_HD189, r_s = rs_HD189, a_s = a_HD189, \
                        rho_s = rho_HD189)

print('HD 189733 b:  S =', S_HD189, 'at rho_s =', rho_HD189, ' g/cm3, B_p =', \
      B_HD189, 'G and f =', nu_HD189, 'MHz')

HD 189733 b:  S = 115.3938 mJy at rho_s = 1.769947362903328e-15  g/cm3, B_p = 217.85714285714286 G and f = 610 MHz


In [80]:
# HAT-P-12b (exoplanet archive - Öztürk & Erdem 2019)

# Fixed parameters for this object. Do not modify
p_HATP12  = 3.213   # orbital period (days)
d_HATP12  = 142.75  # distance to the object (parsec) 
m_HATP12  = 0.211   # ratio of planet mass to jupiter mass 
r_HATP12  = 0.949   # ratio of planet radius to jupiter radius

# Following parameters based on GMRT frequencies
B_HATP12  = 150/2.8 # planetary magnetic field (Gauss): ~ 20-70 G (Cauley et al., Yadav et al.)
nu_HATP12 = 150     # cyclotron frequency

# Following parameters can be tweaked
rs_HATP12 = 1       # ratio of satellite radius to Io radius (Earth radius = 3.5 * Io radius) # changed in Mdot
a_HATP12  = 1.16    # 2.11 # 1.16    # ratio of semimajor axis of satellite to planet radius

# Plasma parameters
t_HATP12  = 1552    # ionization timescale (s)
Md_HATP12 = 10**5.8 * 1000 # mass loss rate (kg/s)

# Calculation of plasma density
rho_HATP12 = plasma_density(Mdot = Md_HATP12, t = t_HATP12, a_s = a_HATP12, r_p = r_HATP12, m_p = m_HATP12).value

# Calculation of radio flux using calculated and fixed plasma density
S_HATP12  = radio_flux(d = d_HATP12, nu_c = nu_HATP12, B_p = B_HATP12, r_s = rs_HATP12, a_s = a_HATP12, \
                        rho_s = rho_HATP12)

print('HAT-P-12b:  S =', S_HATP12, 'at rho_s =', rho_HATP12, ' g/cm3, B_p =', \
      B_HATP12, 'G and f =', nu_HATP12, 'MHz')

HAT-P-12b:  S = 1.0451 mJy at rho_s = 8.641483145774932e-15  g/cm3, B_p = 53.57142857142858 G and f = 150 MHz
